In [9]:
import pandas as pd
import openpyxl

In [19]:
# read in the excel, reading in every sheet
# start to investigate the data
 
filepath = r"https://github.com/data-to-insight/ERN-sessions/raw/main/data/903_xlsx_errors.xlsx"

data_903 = pd.read_excel(filepath, sheet_name = None)
data_903.keys() # to get names of each of the sheets


dict_keys(['ad1', 'episodes', 'header', 'missing', 'oc2', 'oc3', 'placed_for_adoption', 'previous_permenance', 'reviews', 'uasc'])

In [23]:
# data_903["ad1"]
data_903["header"].info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2676 entries, 0 to 2675
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   CHILD   2676 non-null   object 
 1   SEX     2676 non-null   int64  
 2   DOB     2676 non-null   object 
 3   ETHNIC  2676 non-null   object 
 4   UPN     2676 non-null   object 
 5   MOTHER  1 non-null      float64
 6   MC_DOB  1 non-null      object 
dtypes: float64(1), int64(1), object(5)
memory usage: 146.5+ KB


In [33]:
# .notna() and /isna()
# 903 validation rul 102 - date of birth is not a valid date
# read 'header' dataframe as 'header'

header = data_903["header"]
# header.info()
# header.head() # can see that DOB is d/m/y

# header["DOB"] = pd.to_datetime(header["DOB"], format = "%d/%m/%Y") 
# will get error if any rows that cannot convert to date

header["DOB"] = pd.to_datetime(header["DOB"], format = "%d/%m/%Y", errors = "coerce") 
# coerce puts errors as NAs, so helpful because we can ask it to return NaNs which will help us validate

invalid_dobs = header["DOB"].isna()
error_rows = header[invalid_dobs]
# or could do invlaid_dobs = header["DOB"].notna() then error_rows = header[~invalid_dobs]

error_rows



,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB
2675,bad_entry_1,7,NaT,Alien,UPDOG,NaN,NaN


In [42]:
# validation rule 105 - Date of LA's decision that a child should be placed for adoption is not a valid date
# table is placed_for_adoption, column is DATE_PLACED

placed_for_adoption = data_903["placed_for_adoption"]
# placed_for_adoption
# placed_for_adoption.info()
placed_for_adoption["DATE_PLACED"] = pd.to_datetime(placed_for_adoption["DATE_PLACED"], format = "%d/%m/%Y", errors = "coerce")
invalid_adoption_date = placed_for_adoption["DATE_PLACED"].isna()
error_105_rows = placed_for_adoption[invalid_adoption_date]

error_105_rows


,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED
29,bad_entry_1,Not a date,NaT,This also isn't a date,oop


In [46]:
# .isin()
# validation rule 103 - the ethnicity code is either not valid or has not been entered
# allowed ethnicity codes are:

ethnicity_codes = ethncity_codes = [
        "WBRI",
        "WIRI",
        "WOTH",
        "WIRT",
        "WROM",
        "MWBC",
        "MWBA",
        "MWAS",
        "MOTH",
        "AIND",
        "APKN",
        "ABAN",
        "AOTH",
        "BCRB",
        "BAFR",
        "BOTH",
        "CHNE",
        "OOTH",
        "REFU",
        "NOBT",
    ]
valid_ethnicity = header["ETHNIC"].isin(ethnicity_codes)
# can also insert other things as a list - eg. isin(["WBRI"])
error_103_rows = header[~valid_ethnicity]

error_103_rows

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB
2675,bad_entry_1,7,NaT,Alien,UPDOG,NaN,NaN


In [52]:
# 114 - data entry to record the status of former carer(s) of an adopted child is invalid
# code_list = ["0", "1"]
# table is ad1, column is FOSTER_CARE
# when we do an isin, need to check the datatype of the list is the same as the values

ad1 = data_903["ad1"]
code_list = ["0", "1"] # be careful, rows in dataframe are integers
ad1["FOSTER_CARE"] = ad1["FOSTER_CARE"].astype("str")

valid_codes = ad1["FOSTER_CARE"].isin(code_list)
error_114_rows = ad1[~valid_codes]
error_114_rows



,CHILD,DOB,DATE_INT,DATE_MATCH,FOSTER_CARE,NB_ADOPTR,SEX_ADOPTR,LS_ADOPTR
29,bad_entry,datedate,notdate,datenot,777,Q,Will,let's go


In [60]:
# rule 184 - date of decision that a child should be placed for adoption is before the child was born
# from header and adoption so need to merge tables on childID first

df = pd.merge(placed_for_adoption, header, how = "left", on = "CHILD", suffixes=[None, "_header"])
df["DOB"] = pd.to_datetime(df["DOB"], format = "%d/%m/%Y", errors = "coerce")
df["DATE_PLACED"] = pd.to_datetime(df["DATE_PLACED"], format = "%Y=%m-%d", errors = "coerce")

placed_before_birth = (df["DATE_PLACED"] >= df["DOB"]) | df["DATE_PLACED"].isna()
# if no date placed do not want to return error, hence or isna()

error_184_rows = df[~placed_before_birth]
error_184_rows


,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED,SEX,DOB_header,ETHNIC,UPN,MOTHER,MC_DOB


In [68]:
# validation rule 302 - first episode starts before child was born
# tables are header and episodes
# DOB form header and DECOM from episodes

episodes = data_903["episodes"]

df_302 = pd.merge(episodes, header, how = "left", on = "CHILD", suffixes=(None, "_header"))

df_302 = df_302.sort_values(["CHILD", "DECOM"], ascending = True)
df_302.drop_duplicates("CHILD", keep = "first", inplace = True)

df_302["DECOM"] = pd.to_datetime(df_302["DECOM"], format = "%d/%m/%Y", errors = "coerce")

df_302

# episode_before_birth = df_302["DECOM"] >= df_302["DOB"] | df_302["DECOM"].isna()
# error_302_rows = df_302[~episode_before_birth]
# error_302_rows

# to get FIRST episode, not just any episode, could sort rows by DECOM and then drop all rows except top one




,CHILD,DECOM,RNE,LS,CIN,PLACE,PLACE_PROVIDER,DEC,REC,REASON_PLACE_CHANGE,HOME_POST,PL_POST,URN,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB
4801,32,2017-02-12,T,J1,N3,K2,PR1,15/04/2017,E3,NaN,ED8 1OX,WI5 9CC,7876040,2,2008-02-15,WROM,N083906900567,NaN,NaN
4042,982,2017-10-07,T,E1,N7,H5,PR4,15/10/2017,X1,APPRR,EF24 4OP,WB15 6HK,3183884,2,2010-11-21,BOTH,V078668886875,NaN,NaN
437,1243,2017-01-02,L,V4,N5,Z1,PR0,18/05/2017,E9,NaN,EC5 5MZ,WC30 9OS,7270736,1,2002-08-19,OOTH,U000920739425,NaN,NaN
1796,1407,2017-06-07,P,L2,N3,U3,PR5,07/10/2017,X1,CUSTOD,EK9 2HR,WI24 8QE,8380497,2,2003-06-26,AOTH,Y010365886691,NaN,NaN
3851,2431,2018-02-01,P,L2,N3,A3,PR5,NaN,E14,NaN,EZ17 3GY,WT6 3JM,8470650,1,2001-01-19,MWBC,W079251598684,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,997201,2017-06-07,T,L3,N4,A5,PR1,NaN,X1,NaN,EC16 9ZF,WN12 2KE,2276120,1,2006-05-24,WBRI,X075905131541,NaN,NaN
2766,997445,2016-06-20,P,J1,N2,U3,PR3,24/09/2017,E48,NaN,EP2 2VO,WE6 1FZ,3786993,2,2009-06-20,REFU,X078091135433,NaN,NaN
155,998192,2017-07-08,T,C2,N2,U1,PR4,NaN,X1,CREQB,EL26 9FW,WA22 6DB,1505258,2,2007-07-25,AIND,P044466275725,NaN,NaN
156,998777,2015-10-11,P,C2,N2,U1,PR2,17/03/2018,X1,CLOSE,ER24 5VS,WJ7 5PQ,1112628,1,2007-11-16,WBRI,X016147706879,NaN,NaN


In [ ]:
# 302 - First episode starts before child was born.
# tables are header and episodes
# DOB from header and DECOM from episodes
 
episodes = dfs['episodes']
 
episodes['DECOM'] = pd.to_datetime(episodes['DECOM'], format="%Y-%m-%d", errors='coerce')
 
# sorts values
episodes = episodes.sort_values(['CHILD', 'DECOM'], ascending=True)
 
# drops duplicates
episodes.drop_duplicates('CHILD', keep='first', inplace=True)
 
df = pd.merge(episodes, header, how='left', on='CHILD', suffixes=['_epi', '_hdr'])
 
error_rows = df[df['DECOM'] < df['DOB']]
error_rows

In [77]:
# better way of doing age calculations

# rule 188 - child is aged under 4 years at the end of the year but a SDQ score or reason for no SDQ score has not been completed

oc2 = data_903["oc2"]

oc2["DOB"] = pd.to_datetime(oc2["DOB"], format = "%d/%m/%Y", errors = "coerce")

collection_end = pd.to_datetime("31/03/2020", dayfirst = True)

# need to find children who are under 4 at the end of the year and EITHER have a SDQ score or SDQ reason

oc2["4bday"] = oc2["DOB"] + pd.DateOffset(years = 4)

# error mask is standard name for validation rules
error_mask = (oc2["4bday"] > collection_end) & (oc2["SDQ_SCORE"].notna() | oc2["SDQ_REASON"].notna())

error_rows = oc2[error_mask]
error_rows


,CHILD,DOB,SDQ_SCORE,SDQ_REASON,CONVICTED,HEALTH_CHECK,IMMUNISATIONS,TEETH_CHECK,HEALTH_ASSESSMENT,SUBSTANCE_MISUSE,INTERVENTION_RECEIVED,INTERVENTION_OFFERED,4bday
2675,bad_child,2024-12-05,oops,bad,no,yes,maybe,i,don't,know,possibly,yes,2028-12-05


In [83]:
# 189 - Child is aged 17 years or over at the beginning of the year, but an Strengths
#  and Difficulties (SDQ) score or a reason for no Strengths and Difficulties (SDQ) score has been completed.
 
# 01/04/2020
# age is 17

collection_start = pd.to_datetime("01/04/2020", dayfirst= True)

oc2["17bday"] = oc2["DOB"] + pd.DateOffset(years = 17)
error_mask_189 = (oc2["17bday"] >= collection_start) & (oc2["SDQ_SCORE"].notna() | (oc2["SDQ_REASON"].notna())                                   

SyntaxError: incomplete input (4181412631.py, line 10)

In [88]:
# 189 - Child is aged 17 years or over at the beginning of the year, but an Strengths
#  and Difficulties (SDQ) score or a reason for no Strengths and Difficulties (SDQ) score has been completed.
 
# 01/04/2020
# age is 17

collection_start = pd.to_datetime("01/04/2019", dayfirst= True)

over_17_start = oc2["DOB"] + pd.DateOffset(years = 17) < collection_start
sdq_score_reason = oc2["SDQ_SCORE"].notna() | oc2["SDQ_REASON"].notna()

error_mask = over_17_start & sdq_score_reason

error_rows = oc2[error_mask]
error_rows

,CHILD,DOB,SDQ_SCORE,SDQ_REASON,CONVICTED,HEALTH_CHECK,IMMUNISATIONS,TEETH_CHECK,HEALTH_ASSESSMENT,SUBSTANCE_MISUSE,INTERVENTION_RECEIVED,INTERVENTION_OFFERED,4bday,17bday
20,947046,2001-08-22,NaN,SDQ5,0,1,0,1,0,1,0,0,2005-08-22,2018-08-22
33,714655,2001-10-01,NaN,SDQ2,1,1,0,1,1,0,1,0,2005-10-01,2018-10-01
35,819792,2001-04-14,25,NaN,1,1,1,1,0,1,1,1,2005-04-14,2018-04-14
43,197474,2001-03-30,5,NaN,0,1,1,1,0,1,1,0,2005-03-30,2018-03-30
58,552166,2001-11-09,34,NaN,1,1,1,1,1,1,1,1,2005-11-09,2018-11-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2634,97512,2001-12-27,NaN,SDQ1,0,0,1,0,1,0,0,1,2005-12-27,2018-12-27
2653,413076,2001-05-14,32,NaN,1,0,1,0,0,0,1,1,2005-05-14,2018-05-14
2659,482205,2001-04-26,32,NaN,1,0,0,0,0,0,0,0,2005-04-26,2018-04-26
2660,473732,2001-06-17,NaN,SDQ1,0,1,0,1,0,1,0,0,2005-06-17,2018-06-17
